# Homework 1: Information Retrieval
## Instructions
1. Students will form teams of three people each and submit a single homework for each team in the format - ID1_ID2_ID3.ipynb
2. Groups of four are not allowed.
2. **Do not write your names anywhere.**
3. For the code part: 
> **Write your code only in the mentioned sections. Do not change the code of other sections**. Do not use any imports unless we say so.
4. For theoretical questions, if any - write your answer in the markdown cell dedicated to this task, in **English**.


#### Deviation from the aforementioned  instructions will lead to reduced grade
---


## Clarifications
1. The same score for the homework will be given to each member of the team.  
2. The goal of this homework is to test your understanding of the concepts presented in the lectures. \
If a topic was not covered in detail during the lecture, you are asked to study it online on your own. 
Anyhow, we provide here detailed explanations for the code part and if you have problems - ask.
3. Questions can be sent to the forum, you are encouraged to ask questions but do so after you have been thinking about your question. 
4. The length of the empty gaps (where you are supposed to write your code) is a recommendation (the amount of space took us to write the solution) and writing longer code will not harm your grade. We do not expect you to use the programming tricks and hacks we used to make the code shorter.   
Having said that, we do encourage you to write good code and keep that in mind - **extreme** cases may be downgraded.  
We also encourage to use informative variable names - it is easier for us to check and for you to understand. 

For your convenience, , the code has a **DEBUG** mode that you may use in order to debug with toy data.  
It is recommended to solve the code in that mode (with efficiency in mind) and then run the code on all the data.
**Do not forget to file the HW with DEBUG == False**.

Download the "Lyrics" dataset from Moodle and put it in the same directory your script is.


5. We use Python 3.7 for programming.
6. Make sure you have all the packages and functions used in the import section. Most of it is native to Anaconda Python distribution.

### Have fun !

# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline
from tqdm import tqdm
from typing import List,Dict
from IPython.display import Image
from IPython.core.display import HTML 
from pathlib import Path

In [2]:
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download("stopwords")
nltk.download("punkt")
from string import punctuation, ascii_lowercase
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maxpo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maxpo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Debug
**you can change this cell**

In [3]:
#DEBUG = True
DEBUG = False

"""
Recommended to start with a small number to get a feeling for the preprocessing with prints (N_ROWS_FOR_DEBUG = 2)
later increase this number for 5*10**3 in order to see that the code runs at reasonable speed, and change the CHUNK_SIZE accordinaly
When setting Debug == False, our code implements bow.fit() in 15-20 minutes according to the tqdm progress bar. Your solution is not supposed to be much further than that.
"""
N_ROWS_FOR_DEBUG = 2
CHUNCK_SIZE = 1 if DEBUG else 5*10**3

# Config

In [4]:
INPUT_FILE_PATH = Path("lyrics.csv")
BOW_PATH = Path("bow.csv")
N_ROWS = N_ROWS_FOR_DEBUG if DEBUG else None
tqdm_n_iterations = N_ROWS//CHUNCK_SIZE +1 if DEBUG else 363*10**3//CHUNCK_SIZE + 1
COLS = [5]

## 1.1 Bag of words model
### Implement the following methods:

* `preprocess_sentence`: 
    * Lower case the word
    * Ignores it if it's in the stopwords list
    * Removes characters which are not in the allowed symbols
    * Stems it and appends it to the output sentence
    * Discards words with length <= 1
    
    
* `update_counts_and_probabilities`: 

    * Update self.unigram count (the amount of time each word is in the text)
    * Update self.bigram count (two consecutive word occurances)
    * Update self.trigram count (three consecutive word occurances)
    * Update inverted index: a dictionary with words as keys and the values is a dictionary - {'DocID' : word_count}   
    
* `compute_word_document_frequency`:

   * For each word count the number of docs it appears in. For example , for the word 'apple' -
$$\sum_{i \in docs} I(apple \in doc_i), I := Indicator function$$


* `update_inverted_index_with_tf_idf_and_compute_document_norm`:

    * Update the inverted index (which currently hold word counts) with tf idf weighing. We will compute tf by dividing with the number of words in each document. 
    * As we want to calculate the document norm, incrementally update the document norm. pay attention that later we apply sqrt to it to finish the process.

#### The result of this code is a bag of words model that already counts for TF-IDF weighing

In [5]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
allowed_symbols = set(l for l in ascii_lowercase)

In [6]:

def preprocess_sentence(sentence : str) -> List[str]:
    output_sentence = []
    for word in word_tokenize(sentence):
        ### YOUR CODE HERE
        _word = word.lower()  # lowercase
        if (len(_word) > 1 and _word not in stop_words):  # initial check for word's length and not a stop word
            cleaned = ""  # placeholder for the word without forbidden symbols
            for ch in _word:
                if ch in allowed_symbols:
                    cleaned += ch
            if cleaned and len(cleaned) > 1 and cleaned not in stop_words:  # if we still have a relevant word
                _word = stemmer.stem(cleaned)  # stemming word
                output_sentence.append(_word)  # adding processed word to final output
        ### END YOUR CODE
    return output_sentence


def get_data_chuncks() -> List[str]:
    for i ,chunck in enumerate(pd.read_csv(INPUT_FILE_PATH, usecols = COLS, chunksize = CHUNCK_SIZE, nrows = N_ROWS)):
        chunck = chunck.values.tolist()
        yield [chunck[i][0] for i in range(len(chunck))] 

class TfIdf:
    def __init__(self):
        self.unigram_count =  Counter()
        self.bigram_count = Counter()
        self.trigram_count = Counter()
        self.document_term_frequency = Counter()
        self.word_document_frequency = {}
        self.inverted_index = {}
        self.doc_norms = {}
        self.n_docs = -1
        self.sentence_preprocesser = preprocess_sentence
        self.bow_path = BOW_PATH

    def update_counts_and_probabilities(self, sentence :List[str],document_id:int) -> None:
        sentence_len = len(sentence)
        self.document_term_frequency[document_id] = sentence_len
        for i,word in enumerate(sentence):
            ### YOUR CODE HERE
            self.unigram_count[word] += 1  # counting 1 word
            if i < sentence_len - 1:  # we still have atleast another word after the current one 
                self.bigram_count[(word,sentence[i + 1])] += 1
            if i < sentence_len - 2:  # we still have atleast 2 more words after the current one
                self.trigram_count[(word,sentence[i + 1],sentence[i + 2])] += 1
            if word not in self.inverted_index.keys():
                self.inverted_index[word] = {document_id : 1}  # starting to count word's apperances in document
            else:
                if document_id not in self.inverted_index[word].keys():  # in case doc's id was not found
                    self.inverted_index[word][document_id] = 1  # starting to count word's apperances in document
                else:
                    self.inverted_index[word][document_id] += 1  # adding word's apperance in document
            ### END YOUR CODE
        
    def fit(self) -> None:
        for chunck in tqdm(get_data_chuncks(), total = tqdm_n_iterations):
            for sentence in chunck: #sentence is a song (string)
                self.n_docs += 1 
                if not isinstance(sentence, str):
                    continue
                sentence = self.sentence_preprocesser(sentence)
                if sentence:
                    self.update_counts_and_probabilities(sentence,self.n_docs)
        self.save_bow() # bow is 'bag of words'
        self.compute_word_document_frequency()
        self.update_inverted_index_with_tf_idf_and_compute_document_norm()
             
    def compute_word_document_frequency(self):
        for word in self.inverted_index.keys():
            ### YOUR CODE HERE
            self.word_document_frequency[word] = len(self.inverted_index[word].keys())
            ### END YOUR CODE
            
    def update_inverted_index_with_tf_idf_and_compute_document_norm(self):
        ### YOUR CODE HERE
        N = len(self.document_term_frequency.keys()) # number of documents
        for word in self.inverted_index.keys():
            word_doc_weights = {}
            for doc in self.inverted_index[word].keys():
                tf = self.inverted_index[word][doc] / self.document_term_frequency[doc]
                df = self.word_document_frequency[word]
                tmp_arr = np.array([N / df])  # for numpy log10 function usage
                idf = np.log10(tmp_arr)[0]
                w_ij = tf * idf 
                word_doc_weights[doc] = w_ij  # replacing int with weight
                if doc not in self.doc_norms.keys():
                    self.doc_norms[doc] = w_ij ** 2
                else:
                    self.doc_norms[doc] += w_ij ** 2
                    
            self.inverted_index[word] = word_doc_weights
        ### END YOUR CODE
        
        for doc in self.doc_norms.keys():
            self.doc_norms[doc] = np.sqrt(self.doc_norms[doc]) 
            
    def save_bow(self):
        pd.DataFrame([self.inverted_index]).T.to_csv(self.bow_path)
                
tf_idf = TfIdf()
tf_idf.fit()

100%|██████████| 73/73 [19:25<00:00, 15.96s/it]


**You need to run the TfIdf model without the DEBUG mode until this stage**

## 1.11 Bag of words model:

1. What is the computational complexity of this model, as a function of the number of docs in the corpus?
2. How can we make this code better in terms running time (parallelization or other topics you find)? 

### YOUR SOLUTION HERE
1. Computational complexity of this model:
    Words as V, Documents as D:
    O(|V|*|D| + |D|) = O(|V|*|D|)

2. Improving running time using parallelization:
    In the current method we process each term in V one by one using a single processing unit. 
    since each term in V is unique, we can divide V into smaller batches and sent each bath to a seperate processing unit to run in parallel. 
    at the end, we combine all the dictionaries into one.




















### END YOUR SOLUTION HERE


## 1.2 DocumentRetriever
Not this retriever &#8595;


![dsafdsafsdafdsf](https://cdn3-www.dogtime.com/assets/uploads/2019/10/golden-cocker-retriever-mixed-dog-breed-pictures-cover-1.jpg)

### Implement the following methods:

`reduce_query_to_counts`: given a list of words returns a counter object with words as keys and counts as values.

`rank`: given a query and relevant documents calculate the similarity (cosine or inner product simmialrity) between each document and the query.   
Make sure to transform the query word counts to tf idf as well. 

`sort_and_retrieve_k_best`: returns the top k documents.



In [7]:
class DocumentRetriever:
    def __init__(self, tf_idf):
        self.sentence_preprocesser = preprocess_sentence  
        self.vocab = set(tf_idf.unigram_count.keys())
        self.n_docs = tf_idf.n_docs
        self.inverted_index = tf_idf.inverted_index
        self.word_document_frequency = tf_idf.word_document_frequency
        self.doc_norms = tf_idf.doc_norms
        
    def rank(self,query : Dict[str,int],documents: Dict[str,Counter],metric: str ) -> Dict[int, float]:
        result = {}  # key: DocID , value : float , similarity to query
        query_len = np.sum(np.array(list(query.values())))
        ### YOUR CODE HERE
        # query tf-idf transformation  
        # for each relevant doc: cosine similarity score
        N = len(tf_idf.document_term_frequency.keys()) # number of documents
        for term in query.keys():
            tf_query = query[term] / query_len
            idf_query = np.log10([N / self.word_document_frequency[term]])[0]
            q = tf_query * idf_query

            for doc in documents[term].keys():
                if doc not in result:
                    result[doc] = q * documents[term][doc]
                else:
                    result[doc] += q* documents[term][doc]
        
         ### END YOUR CODE

        if metric == 'cosine':
        ### YOUR CODE HERE
            for doc in result.keys(): 
                # for each document, we use the result of the inner product as the numerator and the query_len as the denominator
                # to produce the cosine metric rank
                result[doc] *= self.doc_norms[doc] * (1 / query_len)
        ### END YOUR CODE
        return result
        
    
    def sort_and_retrieve_k_best(self, scores: Dict[str, float],k :int):
        ### YOUR CODE HERE 
        # rank similarity scores, return top k docs
        return list(dict(Counter(scores).most_common(k)).keys())
        ### END YOUR CODE
        pass
    
    def reduce_query_to_counts(self, query : List)->  Counter:
        ### YOUR CODE HERE
        return Counter(query) # returning the Counter object  
        ### END YOUR CODE
        
    def get_top_k_documents(self,query : str, metric: str , k = 5) -> List[str]:
        query = self.sentence_preprocesser(query)
        query = [word for word in query if word in self.vocab] # filter nan 
        query_bow = self.reduce_query_to_counts(query)
        relavant_documents = {word : self.inverted_index.get(word) for word in query}
        ducuments_with_similarity = self.rank(query_bow,relavant_documents, metric)
        return self.sort_and_retrieve_k_best(ducuments_with_similarity,k)
        
dr = DocumentRetriever(tf_idf)

In [8]:
from IPython.display import HTML
query = "Better stop dreaming of the quiet life, 'cause it's the one we'll never know And quit running for that runaway bus 'cause those rosy days are few And stop apologizing for the things you've never done 'Cause time is short and life is cruel but it's up to us to change This town called malice"
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/KT6ZtUbVw1M?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

C:\Users\maxpo\AppData\Roaming\Python\Python39\site-packages\IPython\core\display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [9]:
cosine_top_k = dr.get_top_k_documents(query, 'cosine')
print(cosine_top_k)
inner_product_top_k = dr.get_top_k_documents(query, 'inner_product')
print(inner_product_top_k)

[94798, 66192, 61393, 264880, 128160]
[66192, 128160, 309297, 318295, 274541]


In [10]:
for index, song in enumerate(pd.read_csv(INPUT_FILE_PATH,usecols = [5]).iloc[cosine_top_k]['lyrics']):
    sep = "#"*50
    print(F"{sep}\nsong #{index} \n{song} \n{sep}")

##################################################
song #0 
more important is what i'm not doing. theirs less to
apologize for. non-action 
##################################################
##################################################
song #1 
I hear people talk
I see people walk
They seem so out of touch
I wanna get away so much
All the clockwork toys
Are making too much noise
It's the machinery
It's breaking down, oh can't you see
Run runaway, run runaway
Run runaway with me
Run runaway, run runaway
Run runaway with me
The picture doesn't change
It's just a frozen frame
I wanna break the ice
I wanna go to paradise
There is nowhere to hide
I'll take you for a ride
But not if you kiss and tell
I don't mean on a carousel
Won't you run runaway, run runaway
Run runaway with me
Run runaway, run runaway
Run runaway with me
One day I'm to say my
Three wishes came true
Till then I pretend I'm
Escaping with you, you, you, you
Run runaway, run runaway
Run runaway with me
Run runaway, run

# 1.3 term statistics:
Use "tf_idf" object that we created earlier and answer the following questions:

1. How many unique words we have?
2. How many potential word bigrams we have (depend on the unique words we have)? How many actual word bigrams we have? How do you explain this difference?
3. What is the storage size of the input file "lyrics.csv"? What is the output file (bow.csv) size? how do you explain this difference?  

In [11]:
# 1. 
### YOUR SOLUTION HERE
n_unigr = len(tf_idf.unigram_count.keys())
print(f"1. There are {n_unigr} unique words.")
### END YOUR SOLUTION

"""
### Your verbal solution here
There are 387346 unique words.
### End your verbal solution here
"""

# 2.
### YOUR SOLUTION HERE

n_bigr = len(tf_idf.bigram_count.keys())
print(f"2. Potentially, there are ({n_unigr} choose 2) = {75018268185} unique bigrams while {n_bigr} were found.")
### END YOUR SOLUTION

"""
### Your verbal solution here
Potentially, there are (n_unigr choose 2) unique bigrams, because we are choosing all possible pairs out of our corpus.
While in practice only {n_bigr} were found.
### End your verbal solution here
"""

# 3.
### YOUR SOLUTION HERE
print(f"3. {INPUT_FILE_PATH.stat().st_size*1e-6} (MB), {BOW_PATH.stat().st_size*1e-6} (MB) are the storage sizes of lyrics.csv and bow.csv files respectively.")
### END YOUR SOLUTION

"""
### Your verbal solution here
the storage size of the lyrics.csv file is 324.6 (MB) and the storage size of bow.csv is 193.3 (MB
As expected, the BOW file storage size is smaller than the lyrics file as a result of the preprocessing which was applied, like removing stop words and stemming.

### End your verbal solution here
"""

1. There are 387734 unique words.
2. Potentially, there are (387734 choose 2) = 75018268185 unique bigrams while 6310554 were found.
3. 324.632382 (MB), 193.272012 (MB) are the storage sizes of lyrics.csv and bow.csv files respectively.


'\n### Your verbal solution here\n\n\n\n\n### End your verbal solution here\n'

## 1.4 NgramSpellingCorrector
Now we will implement a Ngarm (character Ngrams) spelling corrector. That is, we have an out of vocabulary word (v) and we want to retrieve the most similar words (in our vocabulary) to this word.
we will model the similarity of two words by-

$$sim(v,w) := prior \cdot likelihood = p(w) \cdot P(v|w) $$ 
$$P(v|w) := JaccardIndex =  \frac{|X \cap Y|}{|X \cup Y|}$$

Where v is an out of vocabulary word (typo or spelling mistake), w is in a vocabulary word, X is the ngram set of v and Y is the ngram set of w.
For example, if n == 3, the set of ngrams for word "banana" is set("ban","ana","nan","ana") = {"ban","ana","nan"}

In order to do it efficently, we will first construct an index from the possible Ngrams we have seen in our corpus to the words that those Ngrams appear in, in order prevent comparing v to all of the words in our corpus.
Then, we will implement a function that computes this similarity.

* Make sure you compute the JaccardIndex efficently!

In [12]:
def get_bigrams(word):
    for ngram in nltk.ngrams(word, 2):
        yield "".join(list(ngram))
    
def get_trigrams(word):
    for ngram in nltk.ngrams(word, 3):
        yield "".join(list(ngram))
        
""" 
for example - get_bigrams is a generator, which is an object we can loop on:
for ngram in get_bigrams(word):
    DO SOMETHING
"""

' \nfor example - get_bigrams is a generator, which is an object we can loop on:\nfor ngram in get_bigrams(word):\n    DO SOMETHING\n'

In [13]:
class NgramSpellingCorrector:
    def __init__(self, unigram_counts: Counter, get_n_gram: callable):
        self.unigram_counts = unigram_counts
        self.ngram_index = {}
        self.get_n_grams = get_n_gram
    
    def build_index(self) -> None:
        ### YOUR CODE HERE
        # for each word that was collected during docs indexing (fit process):
        for word in self.unigram_counts.keys():
            # using word as key refering to list of ngrams values:
            self.ngram_index[word] = list(self.get_n_grams(word))  
        ### END YOUR CODE
        
    def get_top_k_words(self,word:str,k=5) -> List[str]:
        ### YOUR CODE HERE
        word_ngram = list(self.get_n_grams(word))  # getting word's n_grams values
        sim_scores = {}  # to keep resemblance score for each term
        for term in self.unigram_counts.keys():  # for each term in the corpus
            prior = self.unigram_counts[term]  # dividing with (# total terms) is redundant for comparison process
            x_inter_y = list(set(word_ngram) & set(self.ngram_index[term]))  # getting intersection for JI numerator
            temp = word_ngram[:]
            temp.extend(self.ngram_index[term])
            x_unif_y = set(temp)  # denominator for JI
            ji = len(x_inter_y) / len(x_unif_y)  # calculating Jacard Index
            sim_scores[term] = prior * ji  # calculating and storing term's similarity to input word
        return list(dict(Counter(sim_scores).most_common(k)).keys())  # returning list of top k candidate terms
        ### END YOUR CODE

class BigramSpellingCorrector(NgramSpellingCorrector):
    def __init__(self, unigram_counts: Counter):
        super().__init__(unigram_counts, get_bigrams)
        
        
class TrigramSpellingCorrector(NgramSpellingCorrector):
    def __init__(self, unigram_counts: Counter):
        super().__init__(unigram_counts, get_trigrams)
        

In [14]:
# OUR TESTING:
# D ={"one" : 2, "two" : 3, "three" : 4, 'plane': 66666, 'fish': 10, 'dog': 1000000, 'plant': 4555, 'plan': 1}

# out_of_vocab_word = 'plan'
# trigram_spelling_corrector = TrigramSpellingCorrector(D)
# trigram_spelling_corrector.build_index()
# trigram_spelling_corrector.get_top_k_words(out_of_vocab_word, 2)

In [15]:
out_of_vocab_word = 'supercalifragilisticexpialidocious'
bigram_spelling_corrector = BigramSpellingCorrector(tf_idf.unigram_count)
bigram_spelling_corrector.build_index()
bigram_spelling_corrector.get_top_k_words(out_of_vocab_word)

['like', 'caus', 'life', 'still', 'time']

In [16]:
trigram_spelling_corrector = TrigramSpellingCorrector(tf_idf.unigram_count)
trigram_spelling_corrector.build_index()
trigram_spelling_corrector.get_top_k_words(out_of_vocab_word)

['life', 'still', 'call', 'listen', 'hous']

## 1.5 Language model
Calculate the log likelihood of a sentence. Once with a bigram markovian langauge model, and once with a trigram model.
for example - the likelihood of the senetence "spiderman spiderman does whatever a spider can" for the bigram model is: 
$$p(spiderman)\cdot p(spiderman|spiderman) \cdot  p(does|spiderman) \cdot p(whatever|does) \cdot  p(a|whatever) \cdot  p(spider|a) \cdot p(can|spider)$$

And for the trigram model:
$$p(spiderman,spiderman)\cdot p(does|spiderman,spiderman) \cdot  p(whatever|spiderman,does) \cdot p(a|does,whatever) \cdot  p(spider|whatever,a) \cdot  p(can|a, spider)$$

Since we do not want a zero probability sentence use Laplace smoothing, as you have seen in the lecture, or here https://en.wikipedia.org/wiki/Additive_smoothing

In [43]:
# for the probability smoothing
NUMERATOR_SMOOTHING = 1 # alpha in https://en.wikipedia.org/wiki/Additive_smoothing
DENOMINATOR_SMOOTHING = 10**4 # d in https://en.wikipedia.org/wiki/Additive_smoothing
def sentence_log_probabilty(unigrams : Counter, bigrams  : Counter,trigrams : Counter, sentence: str):
    bigram_log_likelilhood, trigram_log_likelilhood = 0, 0
    words_in_sentence = sentence.split()
    n_words = len(words_in_sentence)
    for i, word in enumerate(words_in_sentence):
        ### YOUR CODE HERE
        if i == 0: # starting by calculating the prior probability 
            bigram_log_likelilhood += np.log(unigrams[word] + NUMERATOR_SMOOTHING)/ (len(unigrams.keys()) +DENOMINATOR_SMOOTHING)
            
            ### MAXIM AND DANIEL TEST FOR PRIOR FOR TRIGRAMS
            trigram_log_likelilhood += np.log(bigrams[(word,words_in_sentence[i+1])] + NUMERATOR_SMOOTHING) / (len(bigrams.keys()) + DENOMINATOR_SMOOTHING)
            ### MAXIM AND DANIEL TEST FOR PRIOR FOR TRIGRAMS


        # calculating the conditional probability
        if i < n_words - 1:
            N1 = unigrams[word]
            bigram_log_likelilhood += np.log(bigrams[(word,words_in_sentence[i+1])] + NUMERATOR_SMOOTHING) / (N1 + DENOMINATOR_SMOOTHING)
            
            #trigram_log_likelilhood += np.log(bigrams[(word,words_in_sentence[i+1])] + NUMERATOR_SMOOTHING) / (N1 + DENOMINATOR_SMOOTHING)

            #MAXIM AND DANIEL TEST FOR PRIOR FOR TRIGRAMS
              
        if i < n_words - 2:
            N2 = bigrams[word, words_in_sentence[i+1]]
            trigram_log_likelilhood += np.log(trigrams[word,words_in_sentence[i+1],words_in_sentence[i+2]] + NUMERATOR_SMOOTHING) /  (N2 + DENOMINATOR_SMOOTHING)
        
    
#        bigram_log_likelilhood = np.e**(bigram_log_likelilhood)
#        trigram_log_likelilhood = np.e**(trigram_log_likelilhood)
#         bigram_log_likelilhood = np.log(bigram_log_likelilhood)
#         trigram_log_likelilhood = np.log(trigram_log_likelilhood)
        
        ### END YOUR CODE
    return (bigram_log_likelilhood,trigram_log_likelilhood)
   
sentence = "spider man spider man does whatever a spider can"
bigram_log_likelilhood, trigram_log_likelilhood = sentence_log_probabilty(tf_idf.unigram_count, tf_idf.bigram_count, tf_idf.trigram_count, sentence)
print(F"Bigram log likelihood is {bigram_log_likelilhood}")
print(F"Trigram log likelihood is {trigram_log_likelilhood}")

Bigram log likelihood is 0.0006420015518082552
Trigram log likelihood is 0.00032191705393113106


## 1.51 Language model: B
For each model what is the next word prediciton for the sentnence "like big"?

In [18]:
### YOUR CODE HERE
bigram_potential_next_words = [(k[1], tf_idf.bigram_count[k]) for k in tf_idf.bigram_count.keys() if k[0]=='big']
best_candidate_bi = ('', 0)
for word, cntr in bigram_potential_next_words:
    if cntr > best_candidate_bi[1]:
        best_candidate_bi = (word, cntr)
print(f"Bigram model next word prediction: {best_candidate_bi}")

trigram_potential_next_words = [(k[2], tf_idf.trigram_count[k]) for k in tf_idf.trigram_count.keys() if k[0]=='like' and  k[1]=='big']
best_candidate_tri = ('', 0)
for word, cntr in trigram_potential_next_words:
    if cntr > best_candidate_tri[1]:
        best_candidate_tri = (word, cntr)
print(f"Trigram model next word prediction: {best_candidate_tri}")
### END YOUR CODE

Bigram model next word prediction: ('big', 704)
Trigram model next word prediction: ('deal', 41)


## Testing :

In [19]:
stemmer.stem('running')

'run'

In [20]:
#stop_words

In [21]:
'a' in stop_words

True

In [22]:
string = "Special $#! characters   spaces 888323"
''.join(e for e in string if e in allowed_symbols)

'pecialcharactersspaces'

In [23]:
N = 1e6

df = 300
tmp_arr = np.array([N/df])

idf = np.log10(tmp_arr)
idf[0]

3.5228787452803374

In [24]:
tf_idf.inverted_index['one']

 1038: 0.004561948618469171,
 1040: 0.0034137030478340736,
 1046: 0.004014514784252871,
 1047: 0.0035843882002257773,
 1048: 0.017669519296887638,
 1055: 0.006272679350395111,
 1057: 0.002900660971280976,
 1068: 0.024934874436353234,
 1072: 0.012019505342074464,
 1073: 0.007720220738947829,
 1083: 0.039917050411605245,
 1084: 0.007750028541028708,
 1086: 0.025734069129826093,
 1089: 0.004825137961842393,
 1094: 0.001613550958300993,
 1095: 0.01024110914350222,
 1101: 0.004014514784252871,
 1108: 0.010325398107646273,
 1115: 0.01645292944365931,
 1116: 0.009157196131963665,
 1211: 0.012390477729175526,
 1248: 0.015930614223225677,
 1279: 0.027125099893600478,
 1308: 0.0020908931167983704,
 1320: 0.029272503635177183,
 1321: 0.004079791447411454,
 1325: 0.00487198396147193,
 1327: 0.002819181730514656,
 1329: 0.00880376049178261,
 1331: 0.0059036982121365746,
 1349: 0.017010655865478265,
 1355: 0.0029518491060682873,
 1358: 0.00512055457175111,
 1360: 0.03386477195305336,
 1364: 0.007720

In [25]:
pd.read_csv('bow.csv').iloc[190]

Unnamed: 0                                                 make
0             {3: 4, 6: 1, 8: 10, 9: 6, 11: 1, 15: 1, 23: 1,...
Name: 190, dtype: object

In [26]:
pd.read_csv('bow.csv').iloc[190]

Unnamed: 0                                                 make
0             {3: 4, 6: 1, 8: 10, 9: 6, 11: 1, 15: 1, 23: 1,...
Name: 190, dtype: object

In [27]:
tf_idf.inverted_index['one']

 1038: 0.004561948618469171,
 1040: 0.0034137030478340736,
 1046: 0.004014514784252871,
 1047: 0.0035843882002257773,
 1048: 0.017669519296887638,
 1055: 0.006272679350395111,
 1057: 0.002900660971280976,
 1068: 0.024934874436353234,
 1072: 0.012019505342074464,
 1073: 0.007720220738947829,
 1083: 0.039917050411605245,
 1084: 0.007750028541028708,
 1086: 0.025734069129826093,
 1089: 0.004825137961842393,
 1094: 0.001613550958300993,
 1095: 0.01024110914350222,
 1101: 0.004014514784252871,
 1108: 0.010325398107646273,
 1115: 0.01645292944365931,
 1116: 0.009157196131963665,
 1211: 0.012390477729175526,
 1248: 0.015930614223225677,
 1279: 0.027125099893600478,
 1308: 0.0020908931167983704,
 1320: 0.029272503635177183,
 1321: 0.004079791447411454,
 1325: 0.00487198396147193,
 1327: 0.002819181730514656,
 1329: 0.00880376049178261,
 1331: 0.0059036982121365746,
 1349: 0.017010655865478265,
 1355: 0.0029518491060682873,
 1358: 0.00512055457175111,
 1360: 0.03386477195305336,
 1364: 0.007720

In [28]:
tf_idf.unigram_count['do']

2656

In [29]:
# dr = DocumentRetriever(tf_idf)
query = "Better stop dreaming of the quiet life, 'cause it's the one we'll never know And quit running for that runaway bus 'cause those rosy days are few And stop apologizing for the things you've never done 'Cause time is short and life is cruel but it's up to us to change This town called malice"
query = dr.sentence_preprocesser(query)
query = [word for word in query if word in dr.vocab] # filter nan 
query_bow = dr.reduce_query_to_counts(query)
relavant_documents = {word : dr.inverted_index.get(word) for word in query}
#relavant_documents

In [30]:
lyrics_data = pd.read_csv('lyrics copy.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'lyrics copy.csv'

In [16]:
lyrics_data

,index,song,year,artist,genre,lyrics
0,0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing?\nYou know I'm gonna cu..."
1,1,then-tell-me,2009,beyonce-knowles,Pop,"playin' everything so easy,\nit's like you see..."
2,2,honesty,2009,beyonce-knowles,Pop,If you search\nFor tenderness\nIt isn't hard t...
3,3,you-are-my-rock,2009,beyonce-knowles,Pop,"Oh oh oh I, oh oh oh I\n[Verse 1:]\nIf I wrote..."
4,4,black-culture,2009,beyonce-knowles,Pop,"Party the people, the people the party it's po..."
...,...,...,...,...,...,...
362232,362232,who-am-i-drinking-tonight,2012,edens-edge,Country,"I gotta say\nBoy, after only just a couple of ..."
362233,362233,liar,2012,edens-edge,Country,I helped you find her diamond ring\nYou made m...
362234,362234,last-supper,2012,edens-edge,Country,Look at the couple in the corner booth\nLooks ...
362235,362235,christ-alone-live-in-studio,2012,edens-edge,Country,When I fly off this mortal earth\nAnd I'm meas...


In [20]:
 'paul-weller' in lyrics_data.artist.unique()

False

In [23]:
lyrics_data.year[2000]

2007

In [25]:
lyrics_data[lyrics_data.year == 2000]

,index,song,year,artist,genre,lyrics
13581,13581,self-inflicted,2000,divine-empire,Metal,NaN
13582,13582,birth-of-legends,2000,divine-empire,Metal,NaN
13583,13583,dead-and-martyred,2000,divine-empire,Metal,NaN
13584,13584,murder-suicide,2000,divine-empire,Metal,NaN
13585,13585,war-torn,2000,divine-empire,Metal,NaN
...,...,...,...,...,...,...
357796,357796,all-she-wants-to-do-is-dance,2000,the-eagles,Rock,They're picking up the prisoners\nand putting ...
357797,357797,born-to-boogie,2000,the-eagles,Rock,NaN
357798,357798,long-run-leftovers,2000,the-eagles,Rock,"I used to hurry a lot, I used to worry a lot,\..."
357799,357799,random-victims-part-3,2000,the-eagles,Rock,NaN


In [26]:
pd.read_csv(INPUT_FILE_PATH).memory_usage(deep=True).sum()

403678689

In [28]:
pd.read_csv(INPUT_FILE_PATH).info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362237 entries, 0 to 362236
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   index   362237 non-null  int64 
 1   song    362235 non-null  object
 2   year    362237 non-null  int64 
 3   artist  362237 non-null  object
 4   genre   362237 non-null  object
 5   lyrics  266557 non-null  object
dtypes: int64(2), object(4)
memory usage: 385.0 MB


In [29]:
pd.read_csv(BOW_PATH).info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387346 entries, 0 to 387345
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  387344 non-null  object
 1   0           387346 non-null  object
dtypes: object(2)
memory usage: 225.0 MB


In [74]:
tf_idf.bigram_count[('big','big')]

704

In [63]:
tf_idf.bigram_count[('like','like')]

5319

In [62]:
bigram_potential_next_words = [(k[1], tf_idf.bigram_count[k]/len(tf_idf.bigram_count.keys())) for k in tf_idf.bigram_count.keys() if k[0]=='like']
best_candidate = ('', 0)
for word, proba in bigram_potential_next_words:
    if proba > best_candidate[1]:
        best_candidate = (word, proba)
print(best_candidate)



('like', 0.0008430433627650935)


In [64]:
trigram_potential_next_words = [(k[2], tf_idf.trigram_count[k]/len(tf_idf.trigram_count.keys())) for k in tf_idf.trigram_count.keys() if k[0]=='like' and  k[1]=='big']
best_candidate_tri = ('', 0)
for word, proba in trigram_potential_next_words:
    if proba > best_candidate_tri[1]:
        best_candidate_tri = (word, proba)
print(best_candidate_tri)

('deal', 2.4001661149114066e-06)


In [ ]:
bi_potential_words = [tf_idf.bigram_count[k] for k in tf_idf.bigram_count.keys() if k[0]=='like']
for word in bi_potential_words:

tri_potential_words = [tf_idf.trigram_count[k] for k in tf_idf.trigram_count.keys() if k[0]=='like' and k[1]=='big']
# search_key = ("like", "big")
# tri_potential_words = [value for key, value in tf_idf.trigram_count.items() if search_key == key[:len(search_key)]]

In [109]:
[(k[1], tf_idf.bigram_count[k]) for k in tf_idf.bigram_count.keys() if k[0]=='big']

[('deal', 405),
 ('wide', 75),
 ('ego', 49),
 ('must', 6),
 ('crew', 7),
 ('good', 7),
 ('girl', 255),
 ('bodi', 121),
 ('meet', 14),
 ('ballin', 43),
 ('end', 11),
 ('time', 516),
 ('gun', 222),
 ('kiss', 38),
 ('homi', 108),
 ('littl', 55),
 ('ball', 119),
 ('bill', 39),
 ('ass', 233),
 ('butt', 77),
 ('man', 448),
 ('titi', 4),
 ('lick', 5),
 ('dog', 167),
 ('dogg', 7),
 ('boy', 556),
 ('said', 28),
 ('stuff', 14),
 ('drum', 20),
 ('step', 27),
 ('mistak', 426),
 ('ya', 14),
 ('paper', 15),
 ('black', 437),
 ('daddi', 272),
 ('bright', 81),
 ('head', 112),
 ('collar', 1),
 ('grief', 1),
 ('love', 152),
 ('breast', 28),
 ('let', 24),
 ('oooooo', 1),
 ('ten', 12),
 ('buck', 47),
 ('relief', 5),
 ('discrac', 2),
 ('armi', 2),
 ('big', 704),
 ('citi', 522),
 ('brother', 239),
 ('enough', 397),
 ('bigger', 17),
 ('star', 172),
 ('pot', 9),
 ('digit', 2),
 ('word', 65),
 ('small', 119),
 ('smile', 100),
 ('mama', 33),
 ('freak', 20),
 ('sort', 2),
 ('seem', 2),
 ('evil', 2),
 ('mansion', 

In [98]:
for index, song in enumerate(pd.read_csv(INPUT_FILE_PATH,usecols = [5]).iloc[inner_product_top_k]['lyrics']):
    sep = "#"*50
    print(F"{sep}\nsong #{index} \n{song} \n{sep}")

##################################################
song #0 
I hear people talk
I see people walk
They seem so out of touch
I wanna get away so much
All the clockwork toys
Are making too much noise
It's the machinery
It's breaking down, oh can't you see
Run runaway, run runaway
Run runaway with me
Run runaway, run runaway
Run runaway with me
The picture doesn't change
It's just a frozen frame
I wanna break the ice
I wanna go to paradise
There is nowhere to hide
I'll take you for a ride
But not if you kiss and tell
I don't mean on a carousel
Won't you run runaway, run runaway
Run runaway with me
Run runaway, run runaway
Run runaway with me
One day I'm to say my
Three wishes came true
Till then I pretend I'm
Escaping with you, you, you, you
Run runaway, run runaway
Run runaway with me
Run runaway, run runaway
Run runaway with me
Run runaway, run runaway
Run runaway with me
Run runaway, run runaway
Run runaway with me
Run runaway, run runaway
Run runaway with me
Run runaway, run runaway
Ru